In [ ]:
import pandas as pd
import numpy as np
import os
from glob import glob

# 데이터 경로 설정
train_path = './train/'
output_path = './processed_7_480/'  # 데이터를 저장할 경로

# 데이터 저장 폴더 생성
if not os.path.exists(output_path):
    os.makedirs(output_path)

### 1단계: 데이터 로드 및 초기 설정 ###
def load_data(path):
    # 데이터 로드
    file_list = glob(os.path.join(path, '*.csv'))
    dataframes = [pd.read_csv(file) for file in file_list]
    return dataframes

### 2단계: 학습 데이터 준비 및 전처리 ###
def split_to_weekly(data, window_size=7*24*60, step=480, exclusion_period=30):
    segments = []
    for start in range(0, len(data) - window_size + 1, step):
        segment = data[start:start + window_size]
        next_10_min = data[start + window_size: start + window_size + exclusion_period]

        # anomaly가 포함되었거나 직후 10분 내에 anomaly가 있는 경우 제외
        if segment['anomaly'].sum() == 0 and next_10_min['anomaly'].sum() == 0:
            segments.append(segment)

    return segments

# 학습 데이터 로드 및 일주일 단위로 분할
train_dataframes = load_data(train_path)
train_segments = []

# 모든 데이터프레임을 분할하여 저장
for idx, df in enumerate(train_dataframes):
    # 파일 이름에 따라 다르게 저장
    file_prefix = f"TRAIN_{chr(65 + idx)}"
    segments = split_to_weekly(df)
    for seg_idx, segment in enumerate(segments):
        file_name = f"{file_prefix}_week_{seg_idx + 1}.csv"
        segment.to_csv(os.path.join(output_path, file_name), index=False)

print("데이터 전처리 완료. 정상 데이터만 저장되었습니다.")

In [ ]:
# 기존 전처리된 데이터 경로와 새롭게 가공된 데이터를 저장할 경로 설정
input_path = './processed_7_480/'
output_path = './aggregated_7_480/'

# 데이터 저장 폴더 생성
if not os.path.exists(output_path):
    os.makedirs(output_path)

### 1단계: 데이터 로드 및 초기 설정 ###
def load_data(path):
    # 데이터 로드
    file_list = glob(os.path.join(path, '*.csv'))
    dataframes = [pd.read_csv(file) for file in file_list]
    return dataframes, file_list

### 2단계: M 센서 합치기 ###
def aggregate_m_columns(data):
    # M으로 시작하는 모든 센서 칼럼 선택
    m_columns = [col for col in data.columns if col.startswith('M')]
    
    # M 센서의 값 합산
    data['M_sum'] = data[m_columns].sum(axis=1)

    # 기존 M 센서 칼럼 삭제
    data = data.drop(columns=m_columns)

    return data

# 기존 전처리된 파일 로드
processed_dataframes, file_list = load_data(input_path)

# M 센서 합치기 및 새로운 파일로 저장
for df, file_path in zip(processed_dataframes, file_list):
    # 데이터 재가공
    aggregated_df = aggregate_m_columns(df)
    
    # 파일 이름 설정 및 저장
    file_name = os.path.basename(file_path)  # 기존 파일 이름을 유지하되 경로를 변경하여 저장
    aggregated_df.to_csv(os.path.join(output_path, file_name), index=False)

print("M 센서 합치기 작업 완료. 새로운 데이터가 'aggregated_data' 폴더에 저장되었습니다.")